In [ ]:

from datetime import datetime

catalog_name = "auto_geospatial"
schema_name = "default_v6"
volume_name = "data"

model_name = f"{catalog_name}.{schema_name}.collision_prediction"
model_path = f"/Volumes/{catalog_name}/{schema_name}/{volume_name}/models"

start_date = "2024-01-01"
end_date = "2024-12-31"

checkpoint_suffix = f"_{datetime.now().strftime('%Y%m%d%H%M')}"
multi_node_route = True

In [ ]:

def get_spark():
  try:
    #Get Default Spark Session 
    from pyspark.sql import SparkSession
    return (SparkSession.builder.getOrCreate(), False)
  
  except:
    #Try with Spark Connect if default fails
    from databricks.connect import DatabricksSession
    return (DatabricksSession.builder.getOrCreate(), True)

spark,is_spark_connect = get_spark()


Create Catalog, Schema, Table

In [ ]:
# Check if catalog exists
catalogs = [row.catalog for row in spark.sql("SHOW CATALOGS").collect()]

if catalog_name not in catalogs:
    spark.sql(f"CREATE CATALOG {catalog_name}")

spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog_name}.{schema_name}")
spark.sql(f"CREATE VOLUME IF NOT EXISTS {catalog_name}.{schema_name}.{volume_name}")

Adjust Dashboard Queries with Defined Catalog. Schema

In [ ]:
import glob


try:
    
    # Target string to replace
    original_ref = "auto_geospatial.default_v3."
    new_ref = f"{catalog_name}.{schema_name}."


    # Find all files starting with '3-Collision Analytics.'
    matching_files = glob.glob("3-Collision Analytics.*")

    # Open the first matching file
    if matching_files:
        with open(matching_files[0], "r", encoding="utf-8") as f:
            content = f.read()
    else:
        raise FileNotFoundError("No file starting with '3-Collision Analytics.' was found.")

    # Replace all occurrences of the target string
    updated_content = content.replace(original_ref, new_ref)

    # Write the updated content back to the file
    with open(matching_files[0], "w", encoding="utf-8") as f:
        f.write(updated_content)

    print(f"✅ Replaced '{original_ref}' with '{new_ref}' in {matching_files[0]}")
except Exception as e:
    
    print(f"❌ Error trying to update dashboard file: {e}")
